In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from hw_1_optimization import plot_levels, plot_trajectory

plt.rcParams.update({'font.size': 14})
%config InlineBackend.figure_format = 'retina'

- Домашнее задание выполняется в этом же Jupyter Notebook'e. Можно использовать Google Colab, но прислать нужно выгруженный `.ipynb` ноутбук.

- В названии файла укажите свои фамилию и имя

- Решение каждой задачи/пункта задачи поместите после условия.

- В финальной версии, которая будет отправлена на проверку, должны быть удалены все отладочные артефакты. Под таким артефактами подразумеваются любые выводы ячеек, которые никак не прокоментированы в тексте, а также любой массовый/длинный технический вывод.

- При чистом перезапуске ноутбука (*Run -> Restart Kernel and Run All Cells* или *Kernel -> Restart & Run All* в юпитере) все ячейки должны выполняться без ошибок.


## Пример запуска метода.

### Градиентный спуск в невыпуклом случае.

Будем минимизировать функцию Розенброка - это хорошо известная (невыпуклая) функция для сравнения алгоритмов оптимизации

Двумерная функция Розенброка выглядит так
<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Rosenbrock_function.svg/1920px-Rosenbrock_function.svg.png" width="500"/>
</div>


Минимум функции Розенброка находится в $x^* = (1, \ldots, 1)^\top$, значение функции в минимуме равно нулю.

Реализуем градиентный спуск с постоянным шагом. Шаг грубо (с точостью до порядка) подберём руками - перебором найдём наибольшую степень десятки, при которой метод ещё сходится (не улетает на бесконечность).

Чтобы нарисовать графики сходимости, на каждой итерации будем сохранять норму градиента, значение функции, расстояние до решения и текущую итерацию (на практике мы решения не знаем, но на тестовой задаче можем вычислить и эту метрику).

In [ ]:
from scipy.optimize import rosen, rosen_der
from typing import Callable


def gd(f: Callable, df: Callable, x0: np.ndarray, x_star: np.ndarray, h: float, iters: int) -> tuple:
    x = x0.copy()
    grad_log, func_log, dist_log = [np.empty(iters) for _ in range(3)]
    iter_log = np.empty((iters, x0.shape[0]))
    for i in range(iters):
        grad = df(x)
        grad_log[i], func_log[i], dist_log[i], iter_log[i] = (
            np.linalg.norm(grad),
            f(x),
            np.linalg.norm(x - x_star),
            x.copy(),
        )
        x -= h * grad
    return x, grad_log, func_log, dist_log, iter_log


def nesterov(
    f: Callable,
    df: Callable,
    x0: np.ndarray,
    x_star: np.ndarray,
    h: float,
    momentum: float,
    iters: int,
) -> tuple:
    x = x0.copy()
    y = x0.copy()
    grad_log, func_log, dist_log = [np.empty(iters) for _ in range(3)]
    iter_log = np.empty((iters, x0.shape[0]))
    for i in range(iters):
        grad = df(y)
        grad_log[i], func_log[i], dist_log[i], iter_log[i] = (
            np.linalg.norm(grad),
            f(x),
            np.linalg.norm(x - x_star),
            x.copy(),
        )

        x_old = x.copy()
        x = y - h * grad
        y = x + momentum * (x - x_old)

    return x, grad_log, func_log, dist_log, iter_log

In [ ]:
dim = 2
rosen_solution = np.ones(dim)

x0 = np.zeros(dim)  # начальная точка

_, _, _, _, iter_log_gd = gd(f=rosen, df=rosen_der, x0=x0, x_star=rosen_solution, h=2 * 1e-3, iters=1000)
_, _, _, _, iter_log_nesterov = nesterov(
    f=rosen, df=rosen_der, x0=x0, x_star=rosen_solution, h=2 * 1e-3, momentum=0.9, iters=1000
)

In [ ]:
plot_levels(func=rosen, xrange=[-1.5, 1.5], yrange=[-1, 1.5])
plot_trajectory(history=iter_log_nesterov, color="C2", label="nesterov")
plot_trajectory(history=iter_log_gd, label="gd")

## 1. Траектории градиентных методов на двумерной квадратичной функции.

Рассмотрим траекторию градиентного метода на двумерной квадратичной функции вида
$$
f(x) = \frac12 x^\top A x,
$$
где $A\in\mathbb{S}_{++}^n$ -- симметричнач положительно определенная матрица $2\times 2$. Проверьте, что константа гладкости $f$ равна $L = \lambda_{\max}(A)$, а константа сильной выпуклости равна $\mu = \lambda_{\min}(A)$.

Функция ```generate_random_2d_psd_matrix``` создает случайную матрицу с заданными собственными числами. Функция ```armijo``` вычисляет размер шага по правилу Армихо.

**Задание**.

1. Задайте квадратичную функцию и ее градиент.

1. Меняя число обусловленности матрицы, запускайте из одной и той же точки градиентный метод с постоянным шагом, с выбором шага по Армихо и метод Нестерова. Измерьте количество итераций до достижения заданной точности (по аргументу). Также изобразите траектории с помощью ```plot_trajectory```. Сравните и объясните результаты. Размер шага задавайте методом проб и ошибок.

**Замечание**. Здесь придется переписать функцию ```gd``` из введения так, чтобы можно было задавать шаг по условию Армихо. Также, для ускорения линейного поиска, можно передавать функции ```armijo``` размер шага, вычисленный на предыдущей итерации.

3. Задайте размер шага для градиентного спуска и метода Нестерова согласно теории (шаг $1/L$). Происходят ли теперь осцилляции траектории метода? Объясните результат.

In [ ]:
from hw_1_optimization import generate_random_2d_psd_matrix, armijo

# реализация и запуск методов, построение графиков

## 2. Эффективность градиентных методов в зависимости от числа обусловленности.

Из теории известно, что градиентному спуску нужно $O(\kappa \ln(1/\varepsilon))$ итерация для достижения точности $\varepsilon$, а методу Нестерова - $O(\sqrt\kappa \ln(1/\varepsilon))$, где $\kappa = L/\mu$ - число обусловленности функции. Задача это пункта - измерить данные зависимости на примере. Воспользуйтесь функцией ```generate_random_psd_matrix``` для генерации матрицы с нужным числом обусловленности.

**Задание**.

1. Запускайте градиентный спуск и метод Нестерова с теоретически заданными параметрами на задачах с разным числом обусловленности. Постройте график зависимости числа итераций до достижения нужной точности от числа обусловленности в двойном логарифмическом масштабе. Согласуется ли график с теорией?

**Замечание**. Не сохраняйте историю итераций метода, т.к. это займет слишком много памяти. Измените соответствующее место в функции ```gd```.

In [ ]:
from hw_1_optimization import generate_random_psd_matrix

# запуск методов и построение графиков

## 3. Логистическая регрессия.

Задача [логистической регрессии](http://www.machinelearning.ru/wiki/index.php?title=%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F) c $l_2$ регуляризацией
для бинарной классификации имеет вид

$
f(x) = \frac\alpha2 \|x\|_2^2 + \frac1m \sum_{i=1}^m \log (1 + \exp(- b_i \langle a_i, x \rangle)) \to \min_w
$

где $x$-вектор параметров модели, $a_i$ - вектор признаков $i$-го объекта, $b_i \in \{-1, 1 \}$ - метка класса $i$-го объекта. 

Скачиваем датасет (если команда выдаёт ошибку, можно руками скачать и положить файл `mushrooms.txt` в папку с ноутбуком).

In [ ]:
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/mushrooms
!wget "https://raw.githubusercontent.com/niquepolice/misc-files/refs/heads/master/mushrooms.txt" -O mushrooms.txt

In [ ]:
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

data = load_svmlight_file("mushrooms.txt")
X, y = data[0].toarray(), data[1]
y = 2 * y - 3  # map {1, 2} to {-1, 1}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Задание**. 

1. Выпишите градиент и гессиан для данной задачи. Является ли задача выпуклой? А $\mu$ - сильно выпуклой? Покажите, как можно оценить $\mu$. Покажите, что константа гладкости может быть оценена как $L = \max_i (\|a_i\|_2^2 / 4)$, взяв гессиан функции и использовав факт, что $e^t/(1 + e^t)^2\leq 1/4$.

1. Пользуясь формулой для $L$, численно оцените константу Липшица градиента для обучающей части выборки `X_{train}`, `y_{train}`.

1. Реализуйте функцию логистической регрессии и ее градиент.

1. Задайте $\alpha \approx L / 1000$. Запустите метод градиентного спуска для минимизации лосса на обучающей выборке. Длину шага определите из теории, подставив туда полученное $L$. Постройте графики сходимости по норме градиента от номера итерации (стоит сделать порядка 5000 итераций). Используйте логарифмический масштаб по вертикальной оси, чтобы можно было оценить скорость сходимости метода в окрестности решения. Посчитайте accuracy на тестовой выборке.

1. Сравните градиентный метод с методом Нестерова с параметрами по теории и с методом с выбором шага по Армихо.

1. Сделайте то же самое для $\alpha = 0$. Что изменилось в графике сходимости?

In [ ]:
# реализация и запуск методов, построение графиков